### **Project (Homework): Launch Sequence**

**Mission:** Our control system has successfully passed all simulations in the local environment. Before the final launch into orbit (deployment to a production server), we must perform the last critical adjustments: secure the configuration, improve the appearance, and prepare the startup sequence for the production engines. The goal is to transform our development prototype into a robust, secure, and professional-looking application ready for real-world operation.

`základní struktura naší aplikace`:
- my_project/
    - wsgi.py
    - app/
        - **Procfile**
        - **requirements.txt**
        - **.env**
        - _ _ init _ _.py
        - database.py
        - models/
            - _ _ init _ _.py
            - crewmember.py
        - data/
            - _ _ init _ _.py
            - asteroid.py
            - crew.py
        - templates/
        - static/
        - config/
        - main/ 
            - _ _ init _ _.py
            - routes.py 
            - templates/ 
        - crew/
            - _ _ init _ _.py
            - routes.py 
            - forms.py
            - templates/
        - api/ 
            - _ _ init _ _.py
            - routes.py
        - /instance
            - my_database.db


- The file with our **SQLite DB** is in the default **instance** folder next to our application, which is the **root** folder.


1.  **Secure Configuration:** Modify `config.py` and `app/__init__.py` to load sensitive data (`SECRET_KEY`, `DATABASE_URL`) from environment variables. Create an `.env` file for local development.

2.  **Production Dependencies:** Create the `requirements.txt` and `Procfile` files needed for Gunicorn.

3.  **Appearance Improvement:** Integrate **Bootstrap** into the application, modify `base.html`, and try to improve selected elements (e.g., navigation, footer, or page layout) using Bootstrap.

4.  **(Bonus) Mission Start:** Deploy the application to a service like **PythonAnywhere or Render** or another hosting provider.

### Procfile file
- this file has no suffix (e.g. "file.txt") - it's name is just "Procfile"

### requirements.txt file
- in terminal use `pip freeze > requirements.txt` to get it

### .env file
- before publishing on GitHub .env file needs to be placed into .gitignore 

In [ ]:
# app/config/configuration.py

import os # import OS
from dotenv import load_dotenv # import load_dotenv

load_dotenv()

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True 
    SECRET_KEY = os.environ.get("SECRET_KEY") # loads "SECRET_KEY" from .env

    # Database Config (set up DB connection) - for SQLite
    SQLALCHEMY_TRACK_MODIFICATIONS = False # disable warnings in terminal for session changes
    SQLALCHEMY_DATABASE_URI = os.environ.get("DATABASE_URL") # loads "DATABASE_URL" from .env

class ProdConfig(Config):
    APP_ENV = 'production' # sets "APP_ENV" to "production"
    DEBUG = False # disables debug mode

class DevConfig(Config):
    APP_ENV = 'development' # sets "APP_ENV" to "development"
    DEBUG = True # enables debug mode

In [ ]:
# app/__init__.py

import os # import OS
from flask import Flask
from dotenv import load_dotenv # imports load_dotenv
from .config.configuration import ProdConfig # import configuration
from .database import db  # import 'db' object
from .data import my_crew # import data
from .models import CrewMember  # import table models
from .main import main_bp # import blueprint object from 'main'
from .crew import crew_bp # import blueprint object from 'crew'
from .api import api_bp # import blueprint object from 'api'
from flask_bootstrap import Bootstrap5 # import Bootstrap


load_dotenv() # loads values from .env file


def app_factory():

    app = Flask(__name__)

    # CONFIGURATION
    app.config.from_object(ProdConfig)

    # PLUGIN INITIALIZATION - extensions for "app" object
    db.init_app(app) # connects "Flask app" and "SQLAlchemy db"
    bs = Bootstrap5(app) # connects "Flask app" and "Bootstrap"

    # REGISTRATION (Blueprints)
    app.register_blueprint(main_bp) # register Main BP
    app.register_blueprint(crew_bp) # register Crew BP
    app.register_blueprint(api_bp, url_prefix="/api") # register API BP, with prefix "/api"
    
    # APP CONTEXT
    with app.app_context(): # use "app_context()
        db.create_all() # creates tables (if no tables in DB)
        if not CrewMember.query.first(): # populates database (if DB is empty)
            for crew_member in my_crew:
                    new_crew_member = CrewMember(
                    id=crew_member["id"], 
                    name=crew_member["name"], 
                    specialization=crew_member["specialization"], 
                    status=crew_member["status"])
            db.session.add(new_crew_member)
            db.session.commit()

    return app

## /models

In [ ]:
# app/models/__init__.py

from .crewmember import CrewMember

In [ ]:
# app/models/crewmember.py

from ..database import db 


class CrewMember(db.Model):
    __tablename__ = "crew_members"
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    specialization = db.Column(db.String(100), nullable=False)
    status = db.Column(db.String(100), nullable=False)

    def __repr__(self):
        return f"<CrewMember {self.name}>"

## /data

In [ ]:
# app/data/__init__.py

from .asteroid import my_asteroid
from .crew import my_crew

In [ ]:
# data/asteroid

my_asteroid = {"name": "230 Athamantis",
               "diameter": "118±2 km",
               "mass": "(2.3±1.1)*10**18 kg",
               "density": "2.7±1.3 g/cm3",
               "speed": "19.3 km/s",
               "img": "static/img/Athamantis.jpeg"
}

In [ ]:
# data/crew

my_crew = [
        {
            "id": 1,
            "name": "Dave Fisher",
            "specialization": "AI Engineer",
            "status": "Active"
        },
        {
            "id": 2,
            "name": "Izael Alexander",
            "specialization": "Rocket Scientist",
            "status": "Active"
        },
        {
            "id": 3,
            "name": "Tethra Dyagran",
            "specialization": "Roboticist",
            "status": "Active"
        },
        {
            "id": 4,
            "name": "Mura Lan",
            "specialization": "Astronavigator",
            "status": "Active"
        }
    ]

## /main

In [ ]:
# app/main/__init__.py

from flask import Blueprint

# Blueprint 'main'
main_bp = Blueprint('main', __name__, template_folder='templates')

from . import routes # import AFTER Blueprint

In [ ]:
# app/main/routes.py

from flask import Flask, render_template
from ..data import my_asteroid
from . import main_bp # import blueprint object


@main_bp.route("/")
def homepage():
    print("homepage called")
    return render_template("homepage.html")

@main_bp.route("/mission_briefing")
def mission_briefing():
    return render_template("mission_briefing.html")

@main_bp.route("/target_asteroid")
def target_asteroid():
    print("target_asteroid called")
    return render_template("target_asteroid.html", my_asteroid=my_asteroid)

## /crew

In [ ]:
# app/crew/__init__.py

from flask import Blueprint

# Blueprint 'main'
crew_bp = Blueprint('crew', __name__, template_folder='templates')

from . import routes # import AFTER Blueprint

In [ ]:
# app/crew/routes.py

from flask import render_template, redirect, url_for
from .forms import NewCrewMemberForm
from ..models import CrewMember
from ..database import db
from . import crew_bp # import blueprint object


@crew_bp.route("/crew_members")
def crew_members():
    my_crew = db.session.query(CrewMember).all()
    return render_template("crew_members.html", my_crew=my_crew)

@crew_bp.route("/crew_members/<int:id>")
def crew_members_details(id):
    crew_member = db.session.query(CrewMember).get(id)
    if crew_member:
        return render_template("crew_members_details.html", crew_member=crew_member)
    return "Crew member not found"
    
@crew_bp.route("/onboarding", methods=["GET", "POST"])
def onboarding():
    form = NewCrewMemberForm()
    if form.validate_on_submit():
        db.session.add(CrewMember(
            name=form.name.data,
            specialization=form.specialization.data,
            status="Pending.."
        ))
        db.session.commit()
        return redirect(url_for(".crew_members")) # relative BP path
    return render_template("onboarding.html", form=form)

In [ ]:
# app/crew/forms.py

from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, validators

class NewCrewMemberForm(FlaskForm):
    name = StringField(label="Name", validators=[validators.DataRequired()])
    specialization = StringField(label="Specialization", validators=[validators.DataRequired()])
    
    submit = SubmitField(label="Submit new crew member")

## /api

In [ ]:
# app/api/__init__.py

from flask import Blueprint

# Blueprint 'api'
api_bp = Blueprint('api', __name__)

from . import routes # import AFTER Blueprint

In [ ]:
# app/api/routes.py

from flask import Flask, jsonify
from ..database import db
from ..models import CrewMember
from ..data import my_asteroid
from . import api_bp # import blueprint object

   
@api_bp.route("crew_members")
def api_crew_members():
    all_crew_members = db.session.query(CrewMember).all()
    return jsonify(list({"id":member.id, "name":member.name, "specialization":member.specialization, "status":member.status} for member in all_crew_members)), 200
    
@api_bp.route("target_asteroid")
def api_target_asteroid():
    return jsonify(my_asteroid), 200

## /app

In [ ]:
# app/database.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

## /models

In [ ]:
# app/models/__init__.py

from .crewmember import CrewMember

In [ ]:
# app/models/crewmember.py

from ..database import db # Importujeme 'db' objekt


class CrewMember(db.Model):
    __tablename__ = "crew_members"
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    specialization = db.Column(db.String(100), nullable=False)
    status = db.Column(db.String(100), nullable=False)

    def __repr__(self):
        return f"<CrewMember {self.name}>"

---

In [ ]:
# wsgi.py

from app import app_factory


app = app_factory()

if __name__ == "__main__":
    app.run(host='0.0.0.0')

## /templates

In [ ]:
<!-- header.html -->

<nav>
    <ul>
        {# global blueprint 'main' #}
        <li><a href="{{ url_for('main.homepage') }}">Home</a></li> 
        <li><a href="{{ url_for('main.mission_briefing') }}">Our Mission</a></li>
        <li><a href="{{ url_for('main.target_asteroid') }}">The Asteroid</a></li>

        {# global blueprint 'crew' #}
        <li><a href="{{ url_for('crew.crew_members') }}">Our Crew</a></li>
        <li><a href="{{ url_for('crew.onboarding') }}">New Recruits Onboarding</a></li>
    </ul>
</nav>

In [ ]:
<!-- footer.html -->
 
<footer>
    <br>
    <p>Deep Space Exploration Corp.</p>
    <p>All rights reserved</p>
</footer>

In [ ]:
<!-- base.html -->

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block page_title %} Title {% endblock page_title %}</title> 
    
    {{ bootstrap.load_css() }} <!-- Bootstrap CSS -->
</head>

<body>
    {% include 'header.html' %}
    {% block page_content %} Content {% endblock page_content %} 
    {% include 'footer.html' %} 

    {{ bootstrap.load_js() }} <!-- Bootstrap JS -->
</body>

</html>

## /main/templates

In [ ]:
<!-- homepage.html -->

{% extends "base.html" %} 

{% block page_title %} HomePage {% endblock page_title %}

{% block page_content %}
    <h1>Deep Space Exploration Corp.</h1>
    <p>Welcome to our homepage !</p>
    <p>Our goal is to mine asteroids from the solar system.</p>
{% endblock page_content %}

In [ ]:
<!-- mission_briefing.html -->

{% extends "base.html" %} 

{% block page_title %} Mission Debriefing {% endblock page_title %}

{% block page_content %}
    <h1>Mission</h1>
    <p>This is our mission:</p>
    <p>get resources from chosen asteroids</p>
    <p>make it profitable</p>
    <p>expand world's economy</p>
{% endblock page_content %}

In [ ]:
<!-- target_asteroid.html -->

{% extends "base.html" %} 

{% block page_title %} Target Asteroid {% endblock page_title %}

{% block page_content %}
    <h1>Target asteroid: {{ my_asteroid["name"] }}</h1>
    <p>Mean diameter: {{ my_asteroid["diameter"] }}</p>
    <p>Mass: {{ my_asteroid["mass"] }}</p>
    <p>Mean density: {{ my_asteroid["density"] }}</p>
    <p>Average orbital speed: {{ my_asteroid["speed"] }}</p>
    <img src="{{ my_asteroid['img'] }}" width="800px"/> 
{% endblock page_content %}

## /crew/templates

In [ ]:
<!-- crew_members_details.html -->

{% extends "base.html" %} 

{% block page_title %} {{ crew_member["name"] }} {% endblock page_title %}

{% block page_content %}
    <h1>Crew member {{ crew_member["id"] }}</h1>
    <p>Name: {{ crew_member["name"] }}</p>
    <p>Specialization: {{ crew_member["specialization"] }}</p>
    <p>Status: {{ crew_member["status"] }}</p>
{% endblock page_content %}

In [ ]:
<!-- crew_members.html -->

{% extends "base.html" %} 

{% block page_title %} Our Crew {% endblock page_title %}

{% block page_content %}
    {% for member in my_crew %}

        {# relative path - within the BP #}
        <p><a href="{{ url_for('.crew_members_details', id=member['id']) }}">{{ member["name"] }}</a></p>
        
    {% endfor %}
{% endblock page_content %}

In [ ]:
<!-- onboarding.html -->

{% extends "base.html" %} 

{% block page_title %} Onboarding {% endblock page_title %}

{% block page_content %}
<form method = "POST" action = "{{ url_for('.onboarding') }}">  <!-- relative path - within the BP -->
    <p>New crew member onboarding form</p>
    {{ form.hidden_tag() }} 

    <p>{{ form.name.label }} {{ form.name(size=30) }}</p>
    <p>{{ form.specialization.label }} {{ form.specialization(size=30) }}</p>

    <p>{{ form.submit() }}</p>
</form>
{% endblock page_content %}

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom